# 第一步：把问题与函数定义交给 function call，自动选择合适的函数

In [1]:
function_descriptions = [
  {
      "name": "get_student_score",
      "description": "Get the student score by given his or her name",
      "parameters": {
          "type": "object",
          "properties": {
              "student_name": {
                  "type": "string",
                  "description": "The student's name",
              }
          },
          "required": ["student_name"],
      },
    }
]

In [2]:
user_query = "What's the performance of Lucy in the school this year?"

In [3]:
import os
import openai
# 运行此API配置，需要将根目录中的.env中api_key替换为自己的
from dotenv import load_dotenv, find_dotenv

# read local .env file
load_dotenv(find_dotenv())

openai.api_key = os.environ['OPENAI_API_KEY']
model = 'gpt-3.5-turbo-16k-0613'
response = openai.ChatCompletion.create(
    model=model,
    messages=[{"role": "user", "content": user_query}],
    functions=function_descriptions,
    function_call="auto",
)
response_message = response["choices"][0]["message"]
print(response_message)
name = eval(response_message['function_call']['arguments']).get("student_name")

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_student_score",
    "arguments": "{\n  \"student_name\": \"Lucy\"\n}"
  }
}


# 第二步：调用 function call 选择的函数，得到返回值

In [4]:
import json
import inspect

def get_function_parameter_names(function):
  if function is not None and inspect.isfunction(function):
      parameter_names = inspect.signature(function).parameters.keys()
      return list(parameter_names)
  else:
      return None

# Locate the function and make the call
def cal_function(_function_name, _arguments):
    the_function = globals().get(_function_name)
    parameter_names = get_function_parameter_names(the_function)
    parameter_values = []
    for parameter_name in parameter_names:
      parameter_values.append(_arguments[parameter_name])
    return the_function(*parameter_values)

SCORES = { 'Alex': 90, 'Lucy': 60 }
def get_student_score(student_name):

    """Get the student score by given his or her name"""

    score = {
        "name": student_name,
        "score": SCORES[student_name]
    }
    return json.dumps(score)

function_name = response_message["function_call"]["name"]
arguments = json.loads(response_message["function_call"]["arguments"])
print(function_name, arguments)
returned_value = cal_function(function_name, arguments)
print(returned_value)

get_student_score {'student_name': 'Lucy'}
{"name": "Lucy", "score": 60}


# 第三步：将响应发送回模型以进行总结

In [5]:
second_response = openai.ChatCompletion.create(
    model=model,
    messages=[
        {"role": "user", "content": user_query},
        response_message,
        {
            "role": "function",
            "name": "get_student_score",
            "content": returned_value,
        },
    ],
)
print(user_query)
print (second_response['choices'][0]['message']['content'])

What's the performance of Lucy in the school this year?
Lucy's performance in school this year is a score of 60.
